In [25]:
# ==============================
# STEP 1 — Import Libraries
# ==============================
import tensorflow as tf
import numpy as np
import os
import glob

# ==============================
# STEP 2 — Load and Preprocess Data
# ==============================
# Download Shakespeare text
path_to_file = tf.keras.utils.get_file(
    'shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
)
text = open(path_to_file, 'r').read()
print(f"Length of text: {len(text)} characters")

# Create vocabulary
vocab = sorted(set(text))
vocab_size = len(vocab)
print(f"Unique characters: {vocab_size}")

# Character-to-integer mapping
char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

# Sequence length and dataset
seq_length = 50  # smaller for faster testing
examples_per_epoch = len(text)//seq_length

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Shuffle and batch
BATCH_SIZE = 32
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# ==============================
# STEP 3 — Build Model for Training
# ==============================
embedding_dim = 128
rnn_units = 256

def build_model(vocab_size, embedding_dim, rnn_units, batch_size, model_type='gru', stateful=False):
    RNNLayer = {
        'rnn': tf.keras.layers.SimpleRNN,
        'lstm': tf.keras.layers.LSTM,
        'gru': tf.keras.layers.GRU
    }[model_type]
    
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        RNNLayer(units=rnn_units, return_sequences=True, stateful=stateful,
                 recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE, model_type='gru', stateful=False)
model.summary()

# ==============================
# STEP 4 — Train Model
# ==============================
EPOCHS = 5 

Length of text: 1115394 characters
Unique characters: 65


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_4 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [26]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)


In [27]:
checkpoint_dir = './training_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_epoch_{epoch:02d}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,
    save_freq='epoch'
)

# Train
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/5
683/683 ━━━━━━━━━━━━━━━━━━━━ 70s 98ms/step - loss: 2.1611
Epoch 2/5
683/683 ━━━━━━━━━━━━━━━━━━━━ 62s 89ms/step - loss: 1.6956
Epoch 3/5
683/683 ━━━━━━━━━━━━━━━━━━━━ 62s 89ms/step - loss: 1.5647
Epoch 4/5
683/683 ━━━━━━━━━━━━━━━━━━━━ 62s 90ms/step - loss: 1.4999
Epoch 5/5
683/683 ━━━━━━━━━━━━━━━━━━━━ 63s 91ms/step - loss: 1.4592


In [28]:

def build_stateful_model(vocab_size, embedding_dim, rnn_units, batch_size=1, model_type='gru'):
    RNNLayer = {
        'rnn': tf.keras.layers.SimpleRNN,
        'lstm': tf.keras.layers.LSTM,
        'gru': tf.keras.layers.GRU
    }[model_type]
    
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        RNNLayer(units=rnn_units, return_sequences=True, stateful=True,
                 recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

gen_model = build_stateful_model(vocab_size, embedding_dim, rnn_units, batch_size=1, model_type='gru')


In [30]:
# Build generation model first (batch_size=1, sequence length=None)
gen_model.build(tf.TensorShape([1, None]))

# Then load the latest weights
weights_files = glob.glob(os.path.join(checkpoint_dir, "*.weights.h5"))
weights_files.sort()
latest_checkpoint = weights_files[-1]

print("Loading weights from:", latest_checkpoint)
gen_model.load_weights(latest_checkpoint)


Loading weights from: ./training_checkpoints\ckpt_epoch_05.weights.h5


In [34]:
def generate_text(model, start_string, char2idx, idx2char, generation_length=500, temperature=1.0):
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []

    # ✅ Reset state for all RNN layers
    for layer in model.layers:
        if hasattr(layer, "reset_states"):
            layer.reset_states()
    
    for _ in range(generation_length):
        predictions = model(input_eval)
        predictions = predictions[:, -1, :]
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        text_generated.append(idx2char[predicted_id])
        input_eval = tf.expand_dims([predicted_id], 0)
    
    return start_string + ''.join(text_generated)


In [35]:

# 5d — Generate example text
start_string = "To be, or not to be, "
generated_text = generate_text(gen_model, start_string, char2idx, idx2char,
                               generation_length=500, temperature=0.8)

print(generated_text)

To be, or not to be, we proce of many year
Is this speak a more too: thou hast hine.

LEONTES:
I have the other throudn
Ontention of your styalticts be against not his blows
Lucentio spoke, since of thoughts have lest,
ere the very blood is
me and understain age;
And monoth, is but with a baw the light,
Even in this seats had new thus day,
Ere you shall be gone with her with his loyal?

CORIOLANUS:
Renigon serven to his fault!
For there saw it so purpose?

LEONTES:
O, I cannot never had she may:
Duke of my back is a
